In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import math 

from bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': tex_preamble,
})

In [3]:
def mu_T(x,a,b): 
    with np.errstate(divide='ignore',invalid='ignore'):
        return 1/(np.pi)/(np.sqrt((x-a)*(b-x)))

In [4]:
np.random.seed(0)

kn=23
kk=11

λ = np.array([(-1)**i*math.comb(kn-kk-i,kk-i) for i in range(kk+1)])
mult = np.array([math.comb(kn,i) - math.comb(kn,i-1) if i>0 else 1 for i in range(kk+1)])

idx = np.argsort(λ)
λ,mult = λ[idx], mult[idx]

A = sp.sparse.load_npz(f'matrices/K_{kn}_{kk}.npz')
n = A.shape[0]
n_trials = 1

v = np.sign(np.random.rand(n)-.5) / np.sqrt(n)

k = 12
Q,(α,β) = lanczos_reorth(A,v,k)

In [5]:
a = -kk-.1
b = kk+1.1

kT = 250
sT = 2*kT
αT = np.ones(sT+1)*(a+b)/2
βT = np.ones(sT+1)*(b-a)/4
βT[0] *= np.sqrt(2)
mT = get_chebyshev_moments(A,v,kT,a,b)

In [6]:
θ,ω = get_gq(k,α[:12],β[:11])
xx = np.linspace(a,b,5000)
AQ_T = eval_poly(xx,mT*jackson_weights(2*kT+1),αT,βT)

In [7]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))

ax = fig.add_axes([left, bottom, width, height])

ax.plot(xx,AQ_T*mu_T(xx,a,b),color=colors[3],ls=linestyles[3],lw=.85)
ax.plot(xx,AQ_T*mu_T(xx,a,b),color=colors[3],lw=.2)

ax.plot(λ,mult/sum(mult),ls='None',ms=5,marker='s',color=colors[4],markerfacecolor='None')
ax.plot(θ,ω,ls='None',marker=markers[1],ms=3,color=colors[1])

ax.grid(True,linestyle=':',linewidth=.5)

ax.set_xticks(np.arange(int(a),round(b)+1), minor=True)
ax.set_xticks(λ)

ax.set_ylim(0,20)
ax.set_yscale('symlog',linthreshy=1e-8)

plt.savefig('imgs/ch4_Kneser_smoothed.pdf')
plt.close()